# ViT Transformer

The Vision Transformer (ViT) is a transformer encoder model (BERT-like) pretrained on a large collection of images in a supervised fashion, namely ImageNet-21k, at a resolution of 224x224 pixels.

Images are presented to the model as a sequence of fixed-size patches (resolution 16x16), which are linearly embedded. One also adds a [CLS] token to the beginning of a sequence to use it for classification tasks. One also adds absolute position embeddings before feeding the sequence to the layers of the Transformer encoder.

Note that this model does not provide any fine-tuned heads, as these were zero'd by Google researchers. However, the model does include the pre-trained pooler, which can be used for downstream tasks (such as image classification).

By pre-training the model, it learns an inner representation of images that can then be used to extract features useful for downstream tasks: if you have a dataset of labeled images for instance, you can train a standard classifier by placing a linear layer on top of the pre-trained encoder. One typically places a linear layer on top of the [CLS] token, as the last hidden state of this token can be seen as a representation of an entire image.


https://huggingface.co/google/vit-base-patch16-224-in21k


###Paper
https://arxiv.org/pdf/2010.11929



### Training data
The ViT model was pretrained on ImageNet-21k, a dataset consisting of 14 million images and 21k classes. https://www.image-net.org/

### MLFLOW
A Tool for Managing the Machine Learning Lifecycle
MLflow is an open-source platform, purpose-built to assist machine learning practitioners and teams in handling the complexities of the machine learning process. MLflow focuses on the full lifecycle for machine learning projects, ensuring that each phase is manageable, traceable, and reproducible.


https://mlflow.org/docs/latest/index.html

# ONNX (Open Neural Network Exchange)
- https://onnx.ai/onnx/

### ONNX Runtime
ONNX Runtime is a cross-platform machine-learning model accelerator, with a flexible interface to integrate hardware-specific libraries. ONNX Runtime can be used with models from PyTorch, Tensorflow/Keras, TFLite, scikit-learn, and other frameworks.

https://onnxruntime.ai/docs/



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install mlflow   optimum open_clip_torch --quiet

! pip install psutil pynvml -q

In [ ]:
# Transformers installation

! pip install transformers[torch] -q
! pip install accelerate -U -q
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

In [ ]:
! pip install onnxruntime -q
! pip install optimum[onnxruntime] -q

In [ ]:
path_model ="/content/drive/MyDrive/models/nsfw_pytorch"

In [ ]:
from google.colab import userdata

from transformers import ViTImageProcessor, ViTForImageClassification
import os
import sys
import platform
from PIL import Image

In [ ]:


from datasets import load_dataset
from functools import partial
import os

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed, Trainer, TrainingArguments, BitsAndBytesConfig, \
    DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import load_dataset
from torch import cuda, bfloat16
import transformers
import openai

import torch.nn as nn
from google.colab import userdata
import mlflow
import numpy as np

In [ ]:

from google.colab import output
output.enable_custom_widget_manager()

from transformers.utils import logging
from transformers import pipeline

In [ ]:
logging.set_verbosity_error()

os.environ["TRANSFORMERS_VERBOSITY"] = "error"

In [ ]:


device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'
device


In [ ]:
MLFLOW_TRACKING_URI="databricks"
# Specify the workspace hostname and token
DATABRICKS_HOST="https://adb-2467347032368999.19.azuredatabricks.net/"
DATABRICKS_TOKEN=userdata.get('DATABRCKS_TTOKEN')

In [ ]:


if "MLFLOW_TRACKING_URI" not in os.environ:
    os.environ["MLFLOW_TRACKING_URI"] = MLFLOW_TRACKING_URI
if "DATABRICKS_HOST" not in os.environ:
    os.environ["DATABRICKS_HOST"] = DATABRICKS_HOST
if "DATABRICKS_TOKEN" not in os.environ:
    os.environ["DATABRICKS_TOKEN"] = DATABRICKS_TOKEN

In [ ]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [ ]:

mlflow.set_experiment("/Users/pepe@kk.com/nsfw_onnx")


In [ ]:
mlflow.end_run()

In [ ]:

processor = ViTImageProcessor.from_pretrained(path_model)
model = ViTForImageClassification.from_pretrained(path_model)
model = model.to("cpu")

In [ ]:
image_path = "/content/drive/MyDrive/data/beach.jpg"
image = Image.open(image_path).convert("RGB")

In [ ]:
np.array(image).shape

In [ ]:
np.array(image)

In [ ]:
inputs = processor(images=image, return_tensors="pt")

input_tensor = inputs["pixel_values"]

In [ ]:
path_model_onnx ="/content/drive/MyDrive/models/nsfw_pytorch/onnx"

In [ ]:
torch.onnx.export(
    model=model,
    args=(input_tensor,),
    f=f"{path_model_onnx}/vit_nsfw.onnx",
    opset_version=14,
    do_constant_folding=True,
    input_names=["input"],
    output_names=["output"],
    dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}},
)

In [ ]:
import onnxruntime
from PIL import Image
import numpy as np
import os
import datetime

In [ ]:
ort_sess = onnxruntime.InferenceSession(
   f"{path_model_onnx}/vit_nsfw.onnx", providers=["CPUExecutionProvider"]
)

In [ ]:
input_name = ort_sess.get_inputs()[0].name
output_name = ort_sess.get_outputs()[0].name

img_paths = [image_path]

label_dic = {"drawings": 0, "hentai": 1, "neutral": 2, "porn": 3, "sexy": 4}

In [ ]:
def get_key(dict, value):
    """
    return key given a value. From a dictionary
    """
    for key, val in dict.items():
        if val == value:
            return key
    return "Value not found"


def load_img(path):
    image = Image.open(path).convert("RGB")
    image = image.resize((224, 224), Image.BILINEAR)
    img = np.array(image, dtype=np.float32)
    img /= 255.0
    img = np.transpose(img, (2, 0, 1))
    return img

In [ ]:

time1 = datetime.datetime.now()

inputs = np.array([load_img(path) for path in img_paths])
outputs = ort_sess.run([output_name], {input_name: inputs})[0]

logits = np.array(outputs)

probabilities = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)

predicted_class = np.argmax(probabilities, axis=1)

print("Predicted classes :", predicted_class)
print("\n")
output_probs = {}
for prob, key in zip(probabilities[0], range(0, len(probabilities[0]))):
    label = get_key(label_dic, key)
    output_probs[label] = float(prob)
time2 = datetime.datetime.now()
print(output_probs)
secs = time2 - time1
print(f"Time: {secs}")

In [ ]:

import mlflow
from mlflow.models.signature import infer_signature
from mlflow.pyfunc import PythonModel
import pprint



In [ ]:
class NSFW_Classifier_ONNX(PythonModel):

  def load_context(self, context):
        """
        This method initializes the tokenizer and language model
        using the specified model snapshot directory.
        """

        import onnxruntime
        from PIL import Image
        import numpy as np
        import os
        global label_dic
        label_dic = {"drawings": 0, "hentai": 1, "neutral": 2, "porn": 3, "sexy": 4}
        self.model = onnxruntime.InferenceSession( context.artifacts["snapshot"], providers=["CPUExecutionProvider"])
        self.input_name = self.model.get_inputs()[0].name
        self.output_name = self.model.get_outputs()[0].name

  def get_key(self, dict, value):
      """
      return key given a value. From a dictionary
      """
      for key, val in dict.items():
          if val == value:
              return key
      return "Value not found"


  def load_img(self, path):
      image = Image.open(path).convert("RGB")
      image = image.resize((224, 224), Image.BILINEAR)
      img = np.array(image, dtype=np.float32)
      img /= 255.0
      img = np.transpose(img, (2, 0, 1))
      return img


  def predict(self, context, model_input, params=None):
        """
        This method generates prediction for the given input.
        """
        path_image = model_input["path_image"][0]
        inputs = np.array([load_img(path=path_image)])
        outputs = self.model.run([self.output_name], {self.input_name: inputs})[0]

        logits = np.array(outputs)

        probabilities = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)

        predicted_class = np.argmax(probabilities, axis=1)
        output_probs = {}
        for prob, key in zip(probabilities[0], range(0, len(probabilities[0]))):
            label = self.get_key(label_dic, key)
            output_probs[label] = float(prob)

        return output_probs


In [ ]:
import numpy as np
import pandas as pd

import mlflow
from mlflow.models.signature import ModelSignature
from mlflow.types import ColSpec, DataType, ParamSchema, ParamSpec, Schema

from mlflow.models import infer_signature

model_output= {'drawings': 0.008857411332428455, 'hentai': 0.001359450165182352, 'neutral': 0.9857551455497742, 'porn': 0.001926638651639223, 'sexy': 0.002101368270814419}

model_input= {"path_image":"/content/drive/MyDrive/data/beach.jpg"}


signature = infer_signature(model_input=model_input,model_output=model_output)



# Define input example
input_example = {"path_image":"/content/drive/MyDrive/data/beach.jpg"}

In [ ]:
signature

In [ ]:
input_example

In [ ]:
import datetime
now = datetime.datetime.now()
now.strftime("%Y-%m-%d_%H:%M:%S")

In [ ]:
# Get the current base version of torch that is installed, without specific version modifiers
torch_version = torch.__version__.split("+")[0]

In [ ]:

# Start an MLflow run context and log the PHi3 model wrapper along with the param-included signature to
# allow for overriding parameters at inference time
now = datetime.datetime.now()

description= """Log NSFW fine tunned model with mlflow converted to ONNX to be used only in CPU"""
with mlflow.start_run(run_name=f"nsfw_onnx_log_{now.strftime('%Y-%m-%d_%H:%M:%S')}", description=description) as run:
    model_info = mlflow.pyfunc.log_model(
        "nsfw_image_classification",
        python_model=NSFW_Classifier_ONNX(),
        # NOTE: the artifacts dictionary mapping is critical! This dict is used by the load_context() method in our PHi3() class.
        artifacts={"snapshot": "/content/drive/MyDrive/models/nsfw_pytorch/onnx/vit_nsfw.onnx"},

        pip_requirements=[
            f"torch=={torch_version}",
            f"transformers=={transformers.__version__}",
            "pillow",
            "onnxruntime",
            "numpy",

        ],
        input_example=input_example,
        signature=signature,
    )

In [ ]:
run.to_dictionary()

In [ ]:

model_info.model_uri

In [ ]:
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)

In [ ]:
loaded_model

In [ ]:

time1=  datetime.datetime.now()
response = loaded_model.predict({"path_image":"/content/drive/MyDrive/data/beach.jpg"})
time2=  datetime.datetime.now()
print(time2-time1)

In [ ]:

pprint.pprint(response)

In [ ]:
result = mlflow.register_model(
    model_info.model_uri, "vit_nsfw_onnx"
)

In [ ]:
from mlflow import MlflowClient

client = MlflowClient()

In [ ]:
client.get_model_version(name="vit_nsfw_onnx", version=3)

In [ ]:
import mlflow.pyfunc

model_name = "vit_nsfw_onnx"
model_version = 3

model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_version}")

result= model.predict({"path_image":"/content/drive/MyDrive/data/beach.jpg"})

In [ ]:
pprint.pprint(result)

In [ ]:
f"models:/{model_name}/{model_version}"